An important part of building AI systems is taking the time to understand your data and formatting it for the algorithms. This is often called *preprocessing*. In notebook 1, we began to analyze and understand the data. In this notebook we will do so in a more methodical way.

In [84]:
import pandas as pd
import numpy as np

In [2]:
dat = pd.read_csv("./compas-scores-two-years.csv", index_col=0)
n, p = dat.shape

Preprocessing steps may involve 1) dropping rows that do not meet our inclusion criteria, 2) dropping columns that are not useful for our task, or 3) changing the representation of columns so that our algorithms can handle the data. Let's first consider (1). We should be suspicious of cases where the number of days between arrest date and charge date, *days_b_screening_arrest*, is more than 30. How many of these cases are there?

In [14]:
dat[(dat.days_b_screening_arrest < -30) | (dat.days_b_screening_arrest > 30)].shape[0]

735

There is likely a data quality issue for most of these cases. Therefore we will drop these rows

In [15]:
dat = dat[(dat.days_b_screening_arrest >= -30) & (dat.days_b_screening_arrest <= 30)]

In [23]:
dat.shape

(6172, 52)

Missing data is a common problem when using real-world data. How common is missing data for COMPASS?

In [47]:
dat.isnull().sum()

name                          0
first                         0
last                          0
compas_screening_date         0
sex                           0
dob                           0
age                           0
age_cat                       0
race                          0
juv_fel_count                 0
decile_score                  0
juv_misd_count                0
juv_other_count               0
priors_count                  0
days_b_screening_arrest       0
c_jail_in                     0
c_jail_out                    0
c_case_number                 0
c_offense_date              784
c_arrest_date              5388
c_days_from_compas            0
c_charge_degree               0
c_charge_desc                 5
is_recid                      0
r_case_number              3182
r_charge_degree            3182
r_days_from_arrest         4175
r_offense_date             3182
r_charge_desc              3228
r_jail_in                  4175
r_jail_out                 4175
violent_

This show that all rows have age but 3182 rows are missing r_charge_degree. Sometimes one would drop rows with missing values or impute (fill them in) by using the column average. Before we try either approach, let's first eliminate columns that are not useful as predictors in our model 

Let's consider (2): which columns do we want to include as predictors in our model? Some features, like name or arrest date, are not useful for our task; we should drop these. However, we may later be interested in using these columns for visualizations or analyses: for instance we could ask how risk scores have changed over time as a function of COMPAS screening date. So let's leave *dat* as a dataframe that contains all our information and we'll create a new variable *X* that has the columns we want to use in our model.

The columns that may be informative in predicting who will recommit a crime are the following columns:
- Severity of the current crime: *c_charge_degree*
- history of crime: *priors_count*, *juv_fel_count*, *juv_misd_count*, *juv_other_count*
- demographic information: *age*, *race*, *sex*, 

In [118]:
X = dat[['c_charge_degree', 'priors_count', 'juv_fel_count', 'juv_misd_count', 'juv_other_count', 'age', 'race', 'sex']]

**TODO** Create the outcome variable Y.

**TODO** Now let's check again whether any of our rows have missing values. <span style="color:red">ADD</span>

Now let's consider (3): the representation of each column. What is the type of each variable?

In [119]:
X.dtypes

c_charge_degree    object
priors_count        int64
juv_fel_count       int64
juv_misd_count      int64
juv_other_count     int64
age                 int64
race               object
sex                object
dtype: object

*object* is the pandas data type (dtype) for *string* which is a string of text. Our machine learning algorithms however expect all variables to be numeric. We will need to convert the *object* dtypes to numeric (like *int64*). The most straightforward way to do this is to convert each column to multipe *dummy* columns. 

**TODO** Let's see how many values each object column has and how many occurences of each value there are in the data.

In [70]:
X.race.value_counts()

African-American    3175
Caucasian           2103
Hispanic             509
Other                343
Asian                 31
Native American       11
Name: race, dtype: int64

In [120]:
X = pd.get_dummies(X)

It may cause problems to have a column with mostly zeros, so to avoid that, let's group 'Asian' and 'Native American' into 'Other'.

In [121]:
X['race_Other'] = X['race_Other'] + X['race_Asian'] + X['race_Native American']
X = X.drop(['race_Asian', 'race_Native American'], axis=1)

Save this pandas dataframe to a CSV for future use. 

In [122]:
X.to_csv("X_pandas.csv")

Convert it to a numpy matrix, which is required for the algorithms. Save dat, X, Y to csv for future use.

In [94]:
X = X.values
Y = Y.values

In [114]:
np.savetxt('X.csv', X)
np.savetxt('y.csv', Y)
dat.to_csv("dat.csv")

In [111]:
X.dtype

dtype('int64')